# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
import numpy as np
import pandas as pd
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [7]:
document = ET.parse( './data/mondial_database.xml' )

## The top 10 countries with the lowest mortality rates

In [10]:
country_list_inf = []
country_list = []
for element in document.getroot():
    for subelement in element.getiterator('infant_mortality'):
        country_list.append( element.find('name').text )
        country_list_inf.append( np.float(subelement.text))
        #print element.find('name').text+' : '+subelement.text+','
        #country_list.append(element.find('name').text)
        #country_inf_mortality = np.hstack([country_inf_mortality,np.float(subelement.text)])
    
cim = {'Country':country_list, 'Infant_mortality':country_list_inf}
df = pd.DataFrame(data=cim)
df.sort_values('Infant_mortality',ascending=True).head(10)


,Country,Infant_mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


## The 10 cities with the largest population

In [46]:
city_list = []
census_year_list = []
population_list = []

for element in document.getroot():
    #print element.find('name').text
    for subelement in element.getiterator('city'):
        for subsubelement in subelement.getiterator('population'):
            if (subsubelement.get('measured')=='census'):
                city_list.append(subelement.find('name').text)
                census_year_list.append(np.float(subsubelement.get('year')))
                population_list.append(np.float(subsubelement.text))

city_year_population = {'City':city_list,'Year':census_year_list,'Population':population_list}
df2 = pd.DataFrame(data=city_year_population)
df3 = df2.sort_values('Population',ascending=False)
df3.head()
df3.set_index(['City','Year']).head(12)

Population
City      Year            
Shanghai  2010    22315474
          2000    15758892
Delhi     2001    12877470
Mumbai    2011    12442373
          2001    11914398
Beijing   2010    11716620
Moskva    2010    11612885
São Paulo 2010    11152344
Tianjin   2010    11090314
Guangzhou 2010    11071424
Delhi     2011    11034555
Moskva    2002    10382754

## 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [108]:
# Population requirement per country is for the latest estimates, not just necessary census data
# We can get this information in the following columns
# 1. Country
# 2. Ethnic group 
# 3. Total population
# 4. Percentage population
# 5. Population

Country = []
Ethnic_group = []
Total_population = []
Percent_population = []
Ethnic_population = []

for element in document.getiterator('country'):
    # the country name can be found by element.find('name').text
    for i in range(0,len(element.findall('ethnicgroup'))):
        #print element.find('name').text, element.findall('ethnicgroup')[i].text, np.float(element.findall('population')[-1].text) ,np.float(element.findall('ethnicgroup')[i].attrib['percentage']), np.float(element.findall('population')[-1].text)*np.float(element.findall('ethnicgroup')[i].attrib['percentage'])/100. # all the ethnic groups in a country
        Country.append(element.find('name').text)
        Ethnic_group.append(element.findall('ethnicgroup')[i].text)
        Total_population.append(np.float(element.findall('population')[-1].text))
        Percent_population.append(np.float(element.findall('ethnicgroup')[i].attrib['percentage']))
        Ethnic_population.append(np.float(element.findall('population')[-1].text)*np.float(element.findall('ethnicgroup')[i].attrib['percentage'])/100.)
        
ethnic_populations = {'Country':Country,
                      'Ethnicgroup':Ethnic_group,
                      'Totalpopulation':Total_population,
                      'Percentpopulation':Percent_population,
                      'Ethnicpopulation':Ethnic_population}   

df4 = pd.DataFrame(data=ethnic_populations)

In [144]:
df4.head(10)
df4.groupby(['Ethnicgroup']).sum().sort_values('Ethnicpopulation',ascending=False)['Ethnicpopulation'].head(10)

Ethnicgroup
Han Chinese    1.245059e+09
Indo-Aryan     8.718156e+08
European       4.948722e+08
African        3.183251e+08
Dravidian      3.027137e+08
Mestizo        1.577344e+08
Bengali        1.467769e+08
Russian        1.318570e+08
Japanese       1.265342e+08
Malay          1.219936e+08
Name: Ethnicpopulation, dtype: float64

## name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [150]:
document = ET.parse( './data/mondial_database.xml' )
airport_list = []
elevation_list = []
country_list = []
for element in document.getroot():
    for subelement in element.getiterator('airport'):
        for subsubelement in subelement.getiterator('elevation'):
            airport_list.append(subelement.find('name').text)
            elevation_list.append(subsubelement.text )

In [151]:
airport_elevation = {'Airport':airport_list,'Elevation':elevation_list}

In [152]:
df4 = pd.DataFrame(data=airport_elevation)
df4['Elevation'] = df4['Elevation'].astype(float)

In [149]:
df4.sort_values('Elevation',ascending=False).head(10)

,Airport,Elevation
80,El Alto Intl,4063
219,Lhasa-Gonggar,4005
241,Yushu Batang,3963
813,Juliaca,3827
815,Teniente Alejandro Velasco Astete Intl,3311
82,Juana Azurduy De Padilla,2905
334,Mariscal Sucre Intl,2813
805,Coronel Fap Alfredo Mendivil Duarte,2719
807,Mayor General FAP Armando Revoredo Iglesias Ai...,2677
692,Licenciado Adolfo Lopez Mateos Intl,2581
